# Using the Script-Language Container

A [Script-Language Container](https://github.com/exasol/script-languages-release) for the Exasol consists of a Linux container with a complete Linux distribution and all required libraries, such as a script client. A script client is responsible for the communication with the database and for executing the script code.

## Prerequisites

To run this Notebook you need:
- python3.6+
- Docker 17.05+
- Your notebook user needs the permissions to use docker

## Preparing the Notebook

First we need to install and import a few python packages which we need in the course of this notebook.

In [4]:
!pip install -r requirements.txt

In [5]:
import bash_runner as bash # A helper to run bash with interactive output from python
import importlib
from pathlib import Path
import pyexasol
import requests
import textwrap

## Cloning the Git Repository

To use the [Script-Language Container](https://github.com/exasol/script-languages-release) we need to clone their Git Repository with

```
git clone https://github.com/exasol/script-languages-release --recursive
```

We need to use `--recursive` because the repoistory has submodules

In [8]:
slc_path="script-languages-release"
if not Path(slc_path).exists():
    bash.run("""
    git clone https://github.com/exasol/script-languages-release --recursive
    """)
else:
    bash.run(f"""
    cd {slc_path}
    git fetch
    git reset --hard origin/master 
    git submodule foreach git reset --hard origin/master 
    """)

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 28 (delta 13), reused 10 (delta 9), pack-reused 14
Unpacking objects: 100% (28/28), done.
From https://github.com/exasol/script-languages-release
   3bb2b47..f9da602  master     -> origin/master
 * [new branch]      rebuild/nightly_2021_02_11_01_38_43 -> origin/rebuild/nightly_2021_02_11_01_38_43
 * [new branch]      rebuild/nightly_2021_02_12_01_40_31 -> origin/rebuild/nightly_2021_02_12_01_40_31
 * [new branch]      rebuild/nightly_2021_02_13_01_38_31 -> origin/rebuild/nightly_2021_02_13_01_38_31
 * [new branch]      rebuild/nightly_2021_02_14_01_40_20 -> origin/rebuild/nightly_2021_02_14_01_40_20
 * [new branch]      rebuild/nightly_2021_02_15_01_40_16 -> origin/rebuild/nightly_2021_02_15_01_40_16
Fetching submodule script-languages
From https://github.com/exasol/script-languages
   ed13033..2588f3f  master     -> origin/master
HEAD is now

## Buiding and Exporting a Container

To build and export the container you can use `exaslct`. It first builds a series of docker images and then exports the container as tar.gz package. A container is here by defined as a flavor. For this example, we use `flavors/python3-ds-EXASOL-6.1.0` and export it to the directory `containers`.

A container get builds via a series of docker images which then gets exported into a tar file.

In [9]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python3-ds-EXASOL-6.1.0 --export-path containers 2>&1 | grep -E "CreateImageTask"
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
INFO: Informed scheduler that task   DockerCreateImageTask_6ff5cd9a15   has status   PENDING
INFO: [pid 7069] Worker Worker(salt=262088307, workers=5, host=test-statsmodels, username=jupyter, pid=6980) running   DockerCreateImageTask_6ff5cd9a15(image_name=exasol/script-language-container:python3-ds-EXASOL-6.1.0-release)
INFO: DockerCreateImageTask_6ff5cd9a15(image_name=exasol/script-language-container:python3-ds-EXASOL-6.1.0-release): Write complete_target
INFO: DockerCreateImageTask_6ff5cd9a15(image_name=exasol/script-language-container:python3-ds-EXASOL-6.1.0-release): Time since first_run 0.003627 s
INFO: DockerCreateImageTask_6ff5cd9a15(image_name=exasol/script-language-container:python3-ds-EXASOL-6.1.0-release): Time since creation 0.065469 s
INFO: DockerCreateImageTask_6ff5cd9a15(image_name=exasol/script-language-container:python3-ds-EXASOL-6.1.0-release)

### What to do if something doesn't work?

During the build it can happen that external package repositories might be not available or something is wrong on your machine where you run the build. For these cases, `exaslct` stores many logs to identify the problem after a build.

#### Exaslsct Log

The main log for `exaslct` is stored directly in `exaslct.log`. However, it gets overwritten when you start `exaslct` again.

In [10]:
bash.run(f"""tail {slc_path}/exaslct.log""")

===== Luigi Execution Summary =====

The command took 129.454398 s

Cached container under /home/jupyter/data-science-examples/tutorials/script-languages/script-languages-release/.build_output/cache/exports/python3-ds-EXASOL-6.1.0-release-EYFRS54NWXPTDZOBU2ZWGIXIID77PTCFMM2LLHPQNWDGQAQUNIAA.tar.gz

Copied container to containers/python3-ds-EXASOL-6.1.0_release.tar.gz




#### Build Output Directory

More detailed information about the build or other operations can be found in the `.build_output/jobs/*/outputs` directory. Here each run of `exaslsct` create its own directory under `.build_output/jobs`. The `outputs` directory stores each executed task of `exaslct` it outputs and log files in case it produces these. Especially, the Docker tasks such as build, pull and push store the logs returned by the Docker API. This can be helpful for finding problems during build.

In [11]:
bash.run(f"""
find {slc_path}/.build_output/jobs/*/outputs -type f -name '*log' | tail
""")

script-languages-release/.build_output/jobs/2021_02_08_11_32_56_ExportContainers/outputs/ExportContainers_b5e7f2ab98/ExportFlavorContainer_26a0b4cd24/DockerCreateImageTaskWithDeps_9b04966467/DockerBuildImageTask_7b8c282af4/logs/docker-build.log
script-languages-release/.build_output/jobs/2021_02_08_11_32_56_ExportContainers/outputs/ExportContainers_b5e7f2ab98/ExportFlavorContainer_26a0b4cd24/DockerCreateImageTaskWithDeps_bae7286e00/DockerBuildImageTask_c86365106c/logs/docker-build.log
script-languages-release/.build_output/jobs/2021_02_08_11_32_56_ExportContainers/outputs/ExportContainers_b5e7f2ab98/ExportFlavorContainer_26a0b4cd24/ExportContainerTask_e02ed616fc/logs/extract_release_file.log
script-languages-release/.build_output/jobs/2021_02_08_11_32_56_ExportContainers/outputs/ExportContainers_b5e7f2ab98/ExportFlavorContainer_26a0b4cd24/ExportContainerTask_e02ed616fc/logs/pack_release_file.log
script-languages-release/.build_output/jobs/2021_02_10_15_13_45_ExportContainers/outputs/Ex

## Customizing Script-Language Containers

Sometimes you need very specific dependencies or versions of dependencies in the Exasol UDFs. In such a case you can customize a Script-Language Container.

### Flavors of Containers

We provide several flavors of containers as a starting point. They provide different languages and different sets of dependencies.

In [12]:
bash.run(f"""
find {slc_path}/flavors/  -maxdepth 1 -name '*EXASOL*'
""")

script-languages-release/flavors/fancyr-EXASOL-6.1.0
script-languages-release/flavors/standard-EXASOL-6.1.0
script-languages-release/flavors/python3-ds-EXASOL-6.1.0
script-languages-release/flavors/standard-EXASOL-7.0.0
script-languages-release/flavors/standard-EXASOL-6.2.0
script-languages-release/flavors/python3-ds-cuda-preview-EXASOL-6.1.0


### Flavor Definition

Here you can see how a flavor is structured. A flavor consists for several build steps. The build step `flavor_customization` contains everything you need to customize a flavor. The remaining build steps should be only changed with care, but sometimes some dependencies are defined in other build steps because the script client depends on them.

In [13]:
bash.run(f""" 
find -L {slc_path}/flavors/python3-ds-EXASOL-6.1.0 -maxdepth 2
""")

script-languages-release/flavors/python3-ds-EXASOL-6.1.0
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/base_test_build_run
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/release
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/testconfig
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/flavor_test_build_run
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/base_test_deps
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/language_definition
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_base/build_run
script-languages-release/flavor

### Flavor Customization Build Step

The build step `flavor_customization` consists of a Dockerfile and several package lists which can be modified. We recommend to use the package lists to add new packages to the flavor and only modify the Dockerfile, if you need very specific changes, like adding additonal resources.

In [14]:
bash.run(f""" 
find -L {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization -type f
""")

script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
script-languages-release/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/apt_get_packages


#### Dockerfile

The Dockefile consists of two parts. The first part installs the packages from the package list and should only be change with care. The second part is free for your changes. Read the description in Dockerfile careful to what you can do and what shouldn't do.

In [15]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/Dockerfile
""")

############################################################################################
############################################################################################
# This Dockerfile allows you to extend this flavor by installing packages or adding files. 
# IF you didn't change the lines below, you can add packages and their version to the  
# files in ./packages and they get automatically installed.                                
############################################################################################
############################################################################################

#######################################################################
#######################################################################
# Do not change the following lines unless you know what you are doing 
#######################################################################
###################################################################

#### Package Lists

The package lists have a unified format. Each line consists of the package name and the package version seperated by "|". You can comment out a whole if you your line starts with "#" or you can add a trailing comment to package definition. We usually recommand to pin the version, such that there are no suprises for which version gets installed.

In [16]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.

#tensorflow-probability|0.9.0


We now are going to append the new python package "xgboost" to one of the package lists using bash redirection ">>".

In [17]:
bash.run(f""" 
echo "xgboost|1.3.3" >> {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

As you can see below, we now added a new line to our package list.

In [18]:
bash.run(f""" 
cat {slc_path}/flavors/python3-ds-EXASOL-6.1.0/flavor_customization/packages/python3_pip_packages
""")

# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.

#tensorflow-probability|0.9.0
xgboost|1.3.3


#### Rebuilding the customized Flavor

After you changed your flavor you need to rebuild it for that you again call `./exaslsct export --flavor-path <flavor-path>`. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container.

In [19]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python3-ds-EXASOL-6.1.0 --export-path containers 2>&1 | grep -E "DockerPullImageTask|DockerBuildImageTask|Export"
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
INFO: Informed scheduler that task   ExportContainers_0d1d91f1aa   has status   PENDING
INFO: Informed scheduler that task   ExportFlavorContainer_592772ef6b   has status   PENDING
INFO: [pid 9128] Worker Worker(salt=780730160, workers=5, host=test-statsmodels, username=jupyter, pid=9037) running   ExportFlavorContainer_592772ef6b(flavor_path=flavors/python3-ds-EXASOL-6.1.0, release_goals=["release"], export_path=containers, release_name=)
INFO: [pid 9128] Worker Worker(salt=780730160, workers=5, host=test-statsmodels, username=jupyter, pid=9037) new requirements      ExportFlavorContainer_592772ef6b(flavor_path=flavors/python3-ds-EXASOL-6.1.0, release_goals=["release"], export_path=containers, release_name=)
INFO: Informed scheduler that task   ExportContainerTask_d37b22fd31   has status   PENDING
INFO: Informed scheduler that task   CreateExportDirectory_09cc

Your old container doesn't get lost, because when you change a flavor your container gets a new hash code. If you revert your changes the system automatically uses the existing cached container. Below you can see the content of the cache directory for the containers.

In [20]:
bash.run(f"""
ls -sh {slc_path}/.build_output/cache/exports
""")

total 2.2G
626M python3-ds-EXASOL-6.1.0-release-267G7F6F5QNVCSHCMJLDRSPZSIMTZGWUEIYZW2EJ36RSP4PERTDQ.tar.gz
4.0K python3-ds-EXASOL-6.1.0-release-267G7F6F5QNVCSHCMJLDRSPZSIMTZGWUEIYZW2EJ36RSP4PERTDQ.tar.gz.sha256sum
451M python3-ds-EXASOL-6.1.0-release-EYFRS54NWXPTDZOBU2ZWGIXIID77PTCFMM2LLHPQNWDGQAQUNIAA.tar.gz
4.0K python3-ds-EXASOL-6.1.0-release-EYFRS54NWXPTDZOBU2ZWGIXIID77PTCFMM2LLHPQNWDGQAQUNIAA.tar.gz.sha256sum
4.0K python3-ds-EXASOL-6.1.0-release-EYFRS54NWXPTDZOBU2ZWGIXIID77PTCFMM2LLHPQNWDGQAQUNIAA.tar.gz.sha512sum
466M python3-ds-EXASOL-6.1.0-release-RN5HZ72HNOSOJWNBF2M7TWKIOGWZ4U423PAIU7JYFHMRUZM55QNQ.tar.gz
4.0K python3-ds-EXASOL-6.1.0-release-RN5HZ72HNOSOJWNBF2M7TWKIOGWZ4U423PAIU7JYFHMRUZM55QNQ.tar.gz.sha256sum
612M python3-ds-EXASOL-6.1.0-release-ZU7VCDQ3SQ6SDH2WNNYMWM5PJXM5XBPTKNVQCYTWFTNQZA2RUZ6Q.tar.gz
4.0K python3-ds-EXASOL-6.1.0-release-ZU7VCDQ3SQ6SDH2WNNYMWM5PJXM5XBPTKNVQCYTWFTNQZA2RUZ6Q.tar.gz.sha256sum
4.0K python3-ds-EXASOL-6.1.0-release-ZU7VCDQ3SQ6SDH2WNNYMWM5PJXM5X

## Testing the new Script-Language Container

Now that we have an update container, we need to check if our change was successful. For that we going to upload the container to an Exasol Database and have look into it. In this example, we are going to use a local Docker-DB started by `exaslct`. `Exaslct` uses for this our [integration-test-docker-environment](https://github.com/exasol/integration-test-docker-environment). However, you could also use your own Exasol Database by changing the variables below. However, this notebooks needs to be able to access the BucketFS of your Exasol Database or you need to upload the container, manually. 

In [21]:
DATABASE_HOST="localhost"
DATABASE_PORT=8888
DATABASE_USER="sys"
DATABASE_PASSWORD="exasol"
BUCKETFS_PORT=6666
BUCKETFS_USER="w"
BUCKETFS_PASSWORD="write"
BUCKETFS_NAME="bfsdefault"
BUCKET_NAME="default"
PATH_IN_BUCKET="container"

### Starting a local Docker-DB for Testing

#### Start the environment and forward the database and bucketfs ports to the specified host ports. 

**Note:** The Exasol Integration-Test-Docker-Environment requires Docker with privileged mode

**Note:** Starting the environment can take between 3-5 min.

In [23]:
bash.run(f"""
pushd {slc_path}
./exaslct spawn-test-environment --environment-name test --database-port-forward {DATABASE_PORT} --bucketfs-port-forward {BUCKETFS_PORT} &> integration-test-docker-environment.log
tail integration-test-docker-environment.log
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
    - 1 DockerBuildImageTask_0ca7d8e6de(image_name=exasol/script-language-container:db-test-container)
    - 1 DockerCreateImageTask_0ca7d8e6de(image_name=exasol/script-language-container:db-test-container)
    - 1 DockerTestContainerBuild(caller_output_path=[])
    ...

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====

The command took 316.416816 s


### Upload the Container to the Database

To use our container we need to upload it to the BucketFS of the Database. We can use for this `exaslct upload`, if we have the build machine has access to the BucketFS of the Database, otherwise you need to export the container and transfer it to a machine which has access to the BucketFS of the Database and upload it via curl, as described in our [documentation](https://docs.exasol.com/database_concepts/udf_scripts/adding_new_packages_script_languages.htm).

In [24]:
bash.run(f"""
pushd {slc_path}
./exaslct upload \
    --flavor-path flavors/python3-ds-EXASOL-6.1.0 \
    --database-host {DATABASE_HOST}\
    --bucketfs-port {BUCKETFS_PORT} \
    --bucketfs-username {BUCKETFS_USER} \
    --bucketfs-password {BUCKETFS_PASSWORD} \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --release-name current &> upload.log
tail -n 30 upload.log
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages
    - 1 AnalyzeFlavorBaseDeps_5e0efd3447(flavor_path=flavors/python3-ds-EXASOL-6.1.0)
    - 1 AnalyzeFlavorCustomization_5e0efd3447(flavor_path=flavors/python3-ds-EXASOL-6.1.0)
    - 1 AnalyzeLanguageDeps_5e0efd3447(flavor_path=flavors/python3-ds-EXASOL-6.1.0)
    ...

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====

The command took 72.433567 s

Uploaded .build_output/cache/exports/python3-ds-EXASOL-6.1.0-release-ZU7VCDQ3SQ6SDH2WNNYMWM5PJXM5XBPTKNVQCYTWFTNQZA2RUZ6Q.tar.gz to
http://localhost:6666/default/container/python3-ds-EXASOL-6.1.0-release-current.tar.gz


In SQL, you can activate the languages supported by the python3-ds-EXASOL-6.1.0
flavor by using the following statements:


To activate the flavor only for the current session:

ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=localzmq+p

### Getting the language container activation statement without upload

Sometimes you can't use the `upload` command to upload your container to the database. To get regardless of that, your language activation statement you can use the commnad `generate-language-activation`.

In [25]:
bash.run(f"""
pushd {slc_path}
./exaslct generate-language-activation \
    --flavor-path flavors/python3-ds-EXASOL-6.1.0 \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --container-name current  2>&1 | tail -n 15
""")

~/data-science-examples/tutorials/script-languages/script-languages-release ~/data-science-examples/tutorials/script-languages


In SQL, you can activate the languages supported by the python3-ds-EXASOL-6.1.0
flavor by using the following statements:


To activate the flavor only for the current session:

ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/current?lang=python#buckets/bfsdefault/default/container/current/exaudf/exaudfclient_py3';


To activate the flavor on the system:

ALTER SYSTEM SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/current?lang=python#buckets/bfsdefault/default/container/current/exaudf/exaudfclient_py3';



### Connecting to the database and activate the container

After we have a connection to the database we run the `ALTER SESSION` statement we got from the upload.

In [26]:
def connect():
    con=pyexasol.connect(dsn=f"{DATABASE_HOST}:{DATABASE_PORT}",user=DATABASE_USER,password=DATABASE_PASSWORD)
    con.execute("ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=builtin_python3 PYTHON3_DS=localzmq+protobuf:///bfsdefault/default/container/python3-ds-EXASOL-6.1.0-release-current?lang=python#buckets/bfsdefault/default/container/python3-ds-EXASOL-6.1.0-release-current/exaudf/exaudfclient_py3';")
    con.execute("OPEN SCHEMA TEST")
    return con

### Check if your customization did work

We first create a helper UDF which allows us to run arbitrary shell commands inside of a UDF Instance. 

In [27]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SCALAR SCRIPT execute_shell_command_py3(command VARCHAR(2000000), split_output boolean)
EMITS (lines VARCHAR(2000000)) AS
import subprocess

def run(ctx):
    try:
        p = subprocess.Popen(ctx.command,
                             stdout    = subprocess.PIPE,
                             stderr    = subprocess.STDOUT,
                             close_fds = True,
                             shell     = True)
        out, err = p.communicate()
        if isinstance(out,bytes):
            out=out.decode('utf8')
        if ctx.split_output:
            for line in out.strip().split('\\n'):
                ctx.emit(line)
        else:
            ctx.emit(out)
    finally:
        if p is not None:
            try: p.kill()
            except: pass
/
"""))

<ExaStatement session_id=1691757807740715008 stmt_idx=3>

#### Check with "pip list" if a the package "xgboost" got installed

We use our helper UDF to run `pip list` directly in the container and get the list of currently avaiable packages.

In [28]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('python3 -m pip list', true)""")
for r in rs: 
    print(r[0])

Package              Version
-------------------- ---------------
absl-py              0.11.0
astor                0.8.1
autograd             1.3
autograd-gamma       0.5.0
cached-property      1.5.2
click                7.1.2
cycler               0.10.0
formulaic            0.2.1
future               0.18.2
gast                 0.4.0
gensim               3.8.3
grpcio               1.35.0
h5py                 3.1.0
imbalanced-learn     0.7.0
importlib-metadata   3.4.0
interface-meta       1.2.2
joblib               1.0.0
Keras                2.3.1
Keras-Applications   1.0.8
Keras-Preprocessing  1.1.2
kiwisolver           1.3.1
kmodes               0.10.2
lifelines            0.25.8
lxml                 4.6.2
Markdown             3.3.3
matplotlib           3.3.4
mock                 4.0.3
nltk                 3.5
numpy                1.19.5
pandas               1.1.5
patsy                0.5.1
Pillow               8.1.0
pip                  20.3.4
protobuf             3.14.0
pyasn1     

With running `pip list` directly in the container, we see what currently is available in the container. However, sometimes this might not be what we expected. For these cases, `exaslct` stores information about the flavor the container was build from in the build info.

#### Embedded Build Information of the Container

Here we see an overview about the build information which `exaslct` embedded into the container. `Exaslct` stores all packages lists (as defined in the flavor and what actually got installed), the final Dockerfiles and the image info. The image info describes how the underlying docker images of the container got build.

In [29]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('find /build_info', true)""")
for r in rs: 
    print(r[0])

/build_info
/build_info/image_info
/build_info/image_info/python3-ds-EXASOL-6.1.0-language_deps
/build_info/image_info/python3-ds-EXASOL-6.1.0-build_deps
/build_info/image_info/python3-ds-EXASOL-6.1.0-udfclient_deps
/build_info/image_info/python3-ds-EXASOL-6.1.0-release
/build_info/image_info/python3-ds-EXASOL-6.1.0-flavor_base_deps
/build_info/image_info/python3-ds-EXASOL-6.1.0-build_run
/build_info/image_info/python3-ds-EXASOL-6.1.0-flavor_customization
/build_info/dockerfiles
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-language_deps
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-build_deps
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-udfclient_deps
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-release
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-flavor_base_deps
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-build_run
/build_info/dockerfiles/python3-ds-EXASOL-6.1.0-flavor_customization
/build_info/actual_installed_packages
/build_info/actual_installed_packages/release


The following command shows for example, which python3 package pip found directly after the build of the container image.

In [30]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/actual_installed_packages/release/python3_pip_packages', true)""")
for r in rs: 
    print(r[0])

absl-py|0.11.0
astor|0.8.1
autograd|1.3
autograd-gamma|0.5.0
cached-property|1.5.2
click|7.1.2
cycler|0.10.0
formulaic|0.2.1
future|0.18.2
gast|0.4.0
gensim|3.8.3
grpcio|1.35.0
h5py|3.1.0
imbalanced-learn|0.7.0
importlib-metadata|3.4.0
interface-meta|1.2.2
joblib|1.0.0
Keras|2.3.1
Keras-Applications|1.0.8
Keras-Preprocessing|1.1.2
kiwisolver|1.3.1
kmodes|0.10.2
lifelines|0.25.8
lxml|4.6.2
Markdown|3.3.3
matplotlib|3.3.4
mock|4.0.3
nltk|3.5
numpy|1.19.5
pandas|1.1.5
patsy|0.5.1
Pillow|8.1.0
pip|20.3.4
protobuf|3.14.0
pyasn1|0.4.8
pycurl|7.43.0.6
pyexasol|0.16.1
pygobject|3.26.1
pyparsing|2.4.7
python-apt|1.6.5+ubuntu0.5
python-dateutil|2.8.1
pytz|2020.5
PyYAML|5.4.1
regex|2020.11.13
rsa|4.7
scikit-learn|0.24.1
scipy|1.5.4
seaborn|0.11.1
setuptools|52.0.0
six|1.15.0
smart-open|4.1.2
statsmodels|0.12.1
tensorboard|1.13.1
tensorflow|1.13.1
tensorflow-estimator|1.13.0
tensorflow-hub|0.4.0
termcolor|1.1.0
threadpoolctl|2.1.0
tqdm|4.56.0
typing-extensions|3.7.4.3
ujson|4.0.2
websocket-client|

You could for example compare this to the package list of the flavor-customization build step and check if all your requested packages got installed.

In [31]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/packages/flavor_customization/python3_pip_packages', true)""")
for r in rs: 
    print(r[0])

# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.
None
#tensorflow-probability|0.9.0
xgboost|1.3.3


### Testing the new package

After we checked, if the requested packages actually got installed, we now need to try to import and use it. Importing is usually a good first test if a package got successful installed, because often you might already get errors at this step. However, it only files while you using it. We recommend to have a test suite for each new package to check if it works properly, before you start your UDF development. It usually, easier to debug problems if you have very focused tests.

In [33]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SET SCRIPT test_xgboost(i integer)
EMITS (o VARCHAR(2000000)) AS

def run(ctx):
    import xgboost as xgb
    
    ctx.emit("finish")
/
"""))

rs = con.execute("select test_xgboost(1)")
rs.fetchall()

[('finish',)]